In [620]:
import utils as my
import pandas as pd
import pubchempy as pbp
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import torch
import numpy as np
from torch import nn
from tqdm import tqdm
import plotly

In [621]:
data = pd.read_csv('../data.csv')
smiles = data['smiles']
CI_raw = data['A2780 CS']
smiles_nonan = []
CI = []
for i in range(len(smiles)):
    if not pd.isna(smiles[i]) and not pd.isna(CI_raw[i]):
        smiles_nonan.append(smiles[i])
        CI.append(my.strtonp(CI_raw[i]))
CI.pop(272)
smiles_nonan.pop(272)
CI = np.asarray(CI)
CI = np.log(CI)

In [622]:
print(CI[89], np.argmax(CI[:, 1]))

[0.61049514 0.74378704 0.57428024] 271


In [623]:
fingerprints = []
for i in smiles_nonan:
    molecular = Chem.MolFromSmiles(i)
    fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(molecular, 6, nBits=1024)
    fingerprint = list(fingerprint)
    fingerprints.append(fingerprint)


[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerator
[11:12:31] DEPRECATION WARNING: please use MorganGenerat

In [624]:
features = torch.tensor(fingerprints, dtype=torch.float)
labels = torch.tensor(CI[:,1], dtype=torch.float).unsqueeze(1)
# data_processed = pd.DataFrame({'features': features, 'labels': labels})
print(labels.shape)

torch.Size([340, 1])


In [625]:
import torch
from torchvision import transforms

# 定义标准化变换
transform = transforms.Lambda(lambda x: (x - torch.mean(x)) / torch.std(x))

# 应用变换
features = transform(features)

In [626]:
from torch.utils import data
def load_array(array, batchsize, is_train=True):
    dataset = data.TensorDataset(*array)
    return data.DataLoader(dataset, batch_size=batchsize, shuffle=is_train)
dataloader = load_array((features, labels), 128)
net = nn.Sequential(nn.Linear(1024, 256),
                    nn.Sigmoid(),
                    nn.Linear(256, 1))
trainer = torch.optim.SGD(net.parameters(), lr=0.0001)
loss = nn.MSELoss()
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, 0, std=1)

net.apply(init_weights)
# y_ = net(features)
# l = loss(y_, labels)
# l.backward()
# print(net[0].weight.grad)


Sequential(
  (0): Linear(in_features=1024, out_features=256, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=256, out_features=1, bias=True)
)

In [687]:
num_epochs = 10000
for epoch in tqdm(range(num_epochs)):
    for X, y in dataloader:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
print(f'epoch {epoch + 1}, loss {l:f}')
l = loss(net(features), labels)
print(f'epoch {epoch + 1}, loss {l:f}')

100%|██████████| 10000/10000 [00:41<00:00, 240.32it/s]

epoch 10000, loss 0.000667
epoch 10000, loss 0.000667


In [688]:
import plotly.graph_objects as go
net.train(False)
y_ = net(features)
x = y_.flatten().detach()
y = labels.flatten().detach()
fig = go.Figure()
# fig.add_trace(go.Scatter(x = features.flatten(), y=labels.flatten(), mode='markers'))
# fig.add_trace(go.Scatter(x = features.flatten(), y= x))
fig.add_trace(go.Scatter(x = x, y=y, mode='markers'))

# 显示图表
fig.show()